<a href="https://colab.research.google.com/github/shahabbadihi/IT/blob/master/online-content-based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import sklearn
import pandas as pd
import requests
import io

file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/PreprocessedDataset.csv"
response = requests.get(file_url)

dest = 'local-file.csv'

with open(dest, "wb") as filee:
  filee.write(response.content)

df = pd.read_csv(dest)
df=df.fillna('')
df.head(69772)


,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages,cart_id
0,559850,خاک و کود تی تی بسته 10 کیلوگرمی,TC,466132,2,0,verified,not_recommended,خاک عاری از مواد مقوی,اصلا خاک خوبی نبود. کلی سنگ های درشت داشت و اص...,"[""ندارد""]","[""پر از سنگ ریزه ""]",1
1,112946,توستر فلر مدل TO 220,HA,466139,2,0,verified,not_recommended,خراب شدن اهرم پایین برنده,بعد از دو و نیم سال استفاده، اهرم پایین دهنده ...,"[""زیبایی، سبکی و اندازه مناسب توستر""]","[""خراب شدن اهرم پایین برنده""]",2
2,607671,هدست واقعیت مجازی ایکس پی پروداکت مدل VR BOX,AC,466160,1,2,verified,no_idea,قیمت پایین فدای کیفیت .....,کیفیت لنزش پایینه شرکت فروشنده گارانتی تعویض ن...,"[""ارزون""]","[""کیفیت پایین لنز""]",3
3,757892,کیف پول زنانه آیسا مدل KZP-1,FA,466160,1,0,verified,not_recommended,کیفیت ساخت پایین,اصلا با قیمت فروش ویژه هم ارزش خرید نداره_x000...,,"[""کیفیت ساخت پایین\r"",""چسب کاری داخل و بیرون""]",3
4,601446,اسپیکر قابل حمل مدل ip085 مناسب برای آیفون و ...,IT,466160,2,2,verified,no_idea,بدرد ساختن اسپیکر پرتابل میخوره,دوستان من این مدل برای ساخت اسپیکر پرتابل گرفت...,"[""کیفیت قابل قبول برد و آمپیلی فایر داخلی\r"",""...","[""نداشتن بلوتوث""]",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69767,637008,روسری فراس مدل 623472,FA,6677073,1,13,verified,\N,عالیه,خیلی دوسش دارم‌جنسش فوق العادس,"[""ازهرنظر عالیه مرسی دیجی جان""]",,69766
69768,825863,کنسول بازی سونی مدل Playstation 4 Pro مدل Limi...,AV,6677155,18,36,verified,\N,اطلاعات درباره‌ی دستگاه,یه عده اومدن میگن فقط هاردش 2 تراست کسایی که ...,,,69769
69769,381644,هدفون بی سیم نیا مدل Q8 NEW,AC,6677182,3,2,verified,recommended,خوب هست ارزش خرید دارد,خوبه ولی حرفه ای نیست ولی نسبت به قیمت مناسب هستش,,,69770
69770,87904,شطرنج فدراسیونی آیدین طرح 5,TS,6677262,1,0,verified,recommended,کیفیت صفحه و مهره ها,هم صفحه غمم مهره ها از کیفیت خوبی برخوردار بود...,"[""کیفیت مهره ها وصفحه""]",,69772


In [7]:
from pandas.core.frame import DataFrame
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import random
from scipy.sparse import csr_matrix

dff = DataFrame()
dff["user_id"] = df["user_id"]
dff["product_id"]= df["product_id"]
dff["content"] = df["comment"].str.cat(df["advantages"], sep = "-").str.cat(df["disadvantages"], sep = "-").str.cat(df["title"], sep = "-").str.cat(df["product_title"], sep = "-")


dff.head(100)

file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/fa_stop_words.txt"
response = requests.get(file_url)

stop_words = []
with open("stopwords.txt", "wb") as filee:
  filee.write(response.content)

with open("stopwords.txt", "r") as filee:  
  for line in filee:
    stop_words.append(line)
stop_words = set(stop_words)

tf = TfidfVectorizer(analyzer='word',ngram_range=(1,3),min_df=0, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(dff.loc[0:100000, 'content'].values.astype('U'))
print(tfidf_matrix)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آباد', 'آخ', 'آخر', 'آخرها', 'آخه', 'آدمهاست', 'آرام', 'آره', 'آری', 'آزادانه', 'آسان', 'آسانی', 'آسیب', 'آشنایند', 'آشکارا', 'آقا', 'آقای', 'آقایان', 'آمد', 'آمدن', 'آمده', 'آمرانه', 'آن', 'آنان', 'آنانی', 'آنجا', 'آنرا', 'آنطور', 'آنقدر', 'آنكه', 'آنها', 'آنهاست', 'آنچنان', 'آنچه', 'آنکه', 'آنگاه', 'آهان', 'آهای', 'آور', 'آورد', 'آوردن', 'آورده', 'آوه', 'آيد', 'آی', 'آیا', 'آید', 'آیند', 'اتفاقا', 'اثر', 'اجراست', 'احتراما', 'احتمالا', 'احیانا', 'اخیر', 'اخیرا', 'اری', 'از', 'ازاین', 'ازجمله', 'ازش', 'اساسا', 'است', 'استفاد', 'استفاده', 'اسلامی', 'اش', 'اشتباها', 'اشکارا', 'اصلا', 'اصولا', 'اطلاعند', 'اعلام', 'اغلب', 'افزود', 'افسوس', 'اقل', 'اقلیت', 'اكنون', 'الا', 'الاسف', 'الان', 'البت', 'البته', 'الظاهر', 'الهی', 'الی', 'ام', 'اما', 'امروز', 'امروزه', 'امسال', 'امشب', 'امور', 'امیدوارم

  (0, 804220)	0.12258694874266902
  (0, 1130630)	0.1302236467453938
  (0, 1130662)	0.1201284811034217
  (0, 3720269)	0.12575646478446764
  (0, 1263975)	0.12575646478446764
  (0, 2619404)	0.1302236467453938
  (0, 2676753)	0.1302236467453938
  (0, 276676)	0.1302236467453938
  (0, 2172555)	0.12575646478446764
  (0, 1263752)	0.1302236467453938
  (0, 1796323)	0.1302236467453938
  (0, 1952358)	0.1302236467453938
  (0, 268643)	0.1302236467453938
  (0, 3339889)	0.12575646478446764
  (0, 2871265)	0.12258694874266902
  (0, 2850497)	0.1302236467453938
  (0, 2245948)	0.1302236467453938
  (0, 351748)	0.1302236467453938
  (0, 1483189)	0.1302236467453938
  (0, 1535974)	0.1302236467453938
  (0, 3080425)	0.1302236467453938
  (0, 1952568)	0.1302236467453938
  (0, 3608914)	0.1302236467453938
  (0, 2851206)	0.1302236467453938
  (0, 1356503)	0.1302236467453938
  :	:
  (99999, 2263457)	0.3271847327473835
  (99999, 1746451)	0.06038123021276558
  (99999, 2351192)	0.12019306741611066
  (99999, 732116)	0.083796

In [9]:
def recommendations(index):
  cos_sim = (cosine_similarity(tfidf_matrix[index],tfidf_matrix))
  sorted_sim = pd.Series(cos_sim[0]).sort_values(ascending = False)
  top_10 = list(sorted_sim.iloc[1:11].index)
  return top_10

recommendations(100)

[69771, 38323, 85513, 12047, 24119, 39935, 49315, 85959, 98338, 57622]